In [1]:
import os
import warnings
warnings.filterwarnings("ignore")

import duckdb
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt

from rdkit import Chem
from rdkit.Chem import DataStructs, Descriptors, rdFingerprintGenerator

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report

# Data Load

In [2]:
data_dir = "/home/pervinco/Datasets/leash-bio"
save_dir = f"{data_dir}/split_sets"

train_csv = f"{data_dir}/train.csv"
test_csv = f"{data_dir}/test.csv"

train_parquet = f"{data_dir}/train.parquet"
test_parquet = f'{data_dir}/test.parquet'

os.listdir(data_dir)

['test.parquet',
 'train.csv',
 'test.csv',
 'train.parquet',
 'sample_submission.csv']

In [3]:
RADIUS = 2
FPSIZE = 2048

In [4]:
# data = pd.read_csv(train_csv) ## 전체 데이터가 295246830. 2억 9천 5백 24만 6830개.

In [5]:
con = duckdb.connect()
data = con.query(f"""(SELECT *
                      FROM parquet_scan('{train_parquet}')
                      WHERE binds = 0
                      ORDER BY random()
                      LIMIT 100000)
                      UNION ALL
                      (SELECT *
                      FROM parquet_scan('{train_parquet}')
                      WHERE binds = 1
                      ORDER BY random()
                      LIMIT 100000)""").df()

con.close()

In [6]:
data.shape

(200000, 7)

In [7]:
data.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,238923817,O=C(O)CC1(NC(=O)OCC2c3ccccc3-c3ccccc32)CCCCC1,Cc1c(CN)oc2ccc(F)cc12.Cl,Cc1cnc(F)c(CN)c1,Cc1cnc(F)c(CNc2nc(NCc3oc4ccc(F)cc4c3C)nc(NC3(C...,HSA,0
1,216418580,O=C(Nc1ccccc1C(=O)c1ccccc1C(=O)O)OCC1c2ccccc2-...,N#Cc1c(N)cccc1F,NCCc1ncc(Br)s1,N#Cc1c(F)cccc1Nc1nc(NCCc2ncc(Br)s2)nc(Nc2ccccc...,sEH,0
2,243319498,O=C(O)C[C@@H](Cc1ccc(Br)cc1)NC(=O)OCC1c2ccccc2...,Cc1cc(N)n(C(C)C)n1,Nc1cn2nc(Cl)ccc2n1,Cc1cc(Nc2nc(Nc3cn4nc(Cl)ccc4n3)nc(N[C@@H](CC(=...,HSA,0
3,128384282,O=C(N[C@H](Cc1csc2ccccc12)C(=O)O)OCC1c2ccccc2-...,Nc1ccc(N2CCC(N3CCOCC3)CC2)cc1,NCc1cnc(-c2ccccc2)o1,O=C(N[Dy])[C@@H](Cc1csc2ccccc12)Nc1nc(NCc2cnc(...,sEH,0
4,20182083,CCC(C)C(NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,Cl.NCC1CCN(c2ccc(Br)cc2)C1,NCC1CCCCC(F)(F)C1,CCC(C)C(Nc1nc(NCC2CCN(c3ccc(Br)cc3)C2)nc(NCC2C...,BRD4,0


# Preprocessing

```GetMorganGenerator```을 정의할 때 radius의 설정을 바꿔가면서 학습을 해보는 것도 방법이될 수 있을 것.

In [8]:
data['molecule'] = data['molecule_smiles'].apply(Chem.MolFromSmiles) ## 문자열 화학식을 rdkit 객체로 변환한다.

In [9]:
data.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,molecule
0,238923817,O=C(O)CC1(NC(=O)OCC2c3ccccc3-c3ccccc32)CCCCC1,Cc1c(CN)oc2ccc(F)cc12.Cl,Cc1cnc(F)c(CN)c1,Cc1cnc(F)c(CNc2nc(NCc3oc4ccc(F)cc4c3C)nc(NC3(C...,HSA,0,<rdkit.Chem.rdchem.Mol object at 0x7f639c867120>
1,216418580,O=C(Nc1ccccc1C(=O)c1ccccc1C(=O)O)OCC1c2ccccc2-...,N#Cc1c(N)cccc1F,NCCc1ncc(Br)s1,N#Cc1c(F)cccc1Nc1nc(NCCc2ncc(Br)s2)nc(Nc2ccccc...,sEH,0,<rdkit.Chem.rdchem.Mol object at 0x7f639c867190>
2,243319498,O=C(O)C[C@@H](Cc1ccc(Br)cc1)NC(=O)OCC1c2ccccc2...,Cc1cc(N)n(C(C)C)n1,Nc1cn2nc(Cl)ccc2n1,Cc1cc(Nc2nc(Nc3cn4nc(Cl)ccc4n3)nc(N[C@@H](CC(=...,HSA,0,<rdkit.Chem.rdchem.Mol object at 0x7f639c8675f0>
3,128384282,O=C(N[C@H](Cc1csc2ccccc12)C(=O)O)OCC1c2ccccc2-...,Nc1ccc(N2CCC(N3CCOCC3)CC2)cc1,NCc1cnc(-c2ccccc2)o1,O=C(N[Dy])[C@@H](Cc1csc2ccccc12)Nc1nc(NCc2cnc(...,sEH,0,<rdkit.Chem.rdchem.Mol object at 0x7f639c867580>
4,20182083,CCC(C)C(NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,Cl.NCC1CCN(c2ccc(Br)cc2)C1,NCC1CCCCC(F)(F)C1,CCC(C)C(Nc1nc(NCC2CCN(c3ccc(Br)cc3)C2)nc(NCC2C...,BRD4,0,<rdkit.Chem.rdchem.Mol object at 0x7f639c867430>


In [10]:
fpg = rdFingerprintGenerator.GetMorganGenerator(radius=RADIUS, fpSize=FPSIZE)

def compute_fingerprint(mol):
    if mol is None:
        return None
    return fpg.GetFingerprint(mol)

data['fingerprints'] = data['molecule'].apply(compute_fingerprint)

In [11]:
data.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,molecule,fingerprints
0,238923817,O=C(O)CC1(NC(=O)OCC2c3ccccc3-c3ccccc32)CCCCC1,Cc1c(CN)oc2ccc(F)cc12.Cl,Cc1cnc(F)c(CN)c1,Cc1cnc(F)c(CNc2nc(NCc3oc4ccc(F)cc4c3C)nc(NC3(C...,HSA,0,<rdkit.Chem.rdchem.Mol object at 0x7f639c867120>,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,216418580,O=C(Nc1ccccc1C(=O)c1ccccc1C(=O)O)OCC1c2ccccc2-...,N#Cc1c(N)cccc1F,NCCc1ncc(Br)s1,N#Cc1c(F)cccc1Nc1nc(NCCc2ncc(Br)s2)nc(Nc2ccccc...,sEH,0,<rdkit.Chem.rdchem.Mol object at 0x7f639c867190>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
2,243319498,O=C(O)C[C@@H](Cc1ccc(Br)cc1)NC(=O)OCC1c2ccccc2...,Cc1cc(N)n(C(C)C)n1,Nc1cn2nc(Cl)ccc2n1,Cc1cc(Nc2nc(Nc3cn4nc(Cl)ccc4n3)nc(N[C@@H](CC(=...,HSA,0,<rdkit.Chem.rdchem.Mol object at 0x7f639c8675f0>,"[0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
3,128384282,O=C(N[C@H](Cc1csc2ccccc12)C(=O)O)OCC1c2ccccc2-...,Nc1ccc(N2CCC(N3CCOCC3)CC2)cc1,NCc1cnc(-c2ccccc2)o1,O=C(N[Dy])[C@@H](Cc1csc2ccccc12)Nc1nc(NCc2cnc(...,sEH,0,<rdkit.Chem.rdchem.Mol object at 0x7f639c867580>,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,20182083,CCC(C)C(NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,Cl.NCC1CCN(c2ccc(Br)cc2)C1,NCC1CCCCC(F)(F)C1,CCC(C)C(Nc1nc(NCC2CCN(c3ccc(Br)cc3)C2)nc(NCC2C...,BRD4,0,<rdkit.Chem.rdchem.Mol object at 0x7f639c867430>,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [12]:
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_encoded = onehot_encoder.fit_transform(data['protein_name'].values.reshape(-1, 1)).astype(np.uint8)
protein_columns = onehot_encoder.get_feature_names_out(['protein'])
protein_encoded_df = pd.DataFrame(protein_encoded, columns=protein_columns)
data = pd.concat([data, protein_encoded_df], axis=1)

In [13]:
data.to_csv("./train.csv", index=False)

# Training

In [14]:
def convert_to_numpy_array(explicit_bit_vect):
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(explicit_bit_vect, arr)
    return arr

protein_onehot = onehot_encoder.fit_transform(data['protein_name'].values.reshape(-1, 1))
fingerprints = [convert_to_numpy_array(fp) for fp in data['fingerprints'].tolist()]
X = [np.concatenate([fp, protein]) for fp, protein in zip(fingerprints, protein_onehot.tolist())]
y = data['binds'].tolist()

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
# model = RandomForestClassifier(n_estimators=100, random_state=42)
# model.fit(X_train, y_train)

model = lgb.LGBMClassifier(
    n_estimators=100,
    learning_rate=0.1,
    random_state=42,
    reg_alpha=0.1,   # L1 규제
    reg_lambda=0.3   # L2 규제
)
model.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 79993, number of negative: 80007
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.120252 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4078
[LightGBM] [Info] Number of data points in the train set: 160000, number of used features: 2039
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499956 -> initscore=-0.000175
[LightGBM] [Info] Start training from score -0.000175


LGBMClassifier(random_state=42, reg_alpha=0.1, reg_lambda=0.3)

In [17]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
y_pred_proba = model.predict_proba(X_test)[:, 1]

print(f"Validation Accuracy: {accuracy:.4f}")
print(classification_report(y_test, y_pred))
print(y_pred_proba)

Validation Accuracy: 0.8982
              precision    recall  f1-score   support

           0       0.87      0.93      0.90     19993
           1       0.93      0.87      0.89     20007

    accuracy                           0.90     40000
   macro avg       0.90      0.90      0.90     40000
weighted avg       0.90      0.90      0.90     40000

[0.95889501 0.76354902 0.85683616 ... 0.39445076 0.65001904 0.15397274]


# Evaluation

In [18]:
def convert_to_numpy_array(explicit_bit_vect):
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(explicit_bit_vect, arr)
    return arr

In [19]:
chunk_size = 1000
fpg = rdFingerprintGenerator.GetMorganGenerator(radius=RADIUS, fpSize=FPSIZE)

submission_chunks = []
for chunk in pd.read_csv(test_csv, chunksize=chunk_size):
    chunk['molecule'] = chunk['molecule_smiles'].apply(Chem.MolFromSmiles)
    chunk['fingerprints'] = chunk['molecule'].apply(compute_fingerprint)
    
    protein_onehot_test = onehot_encoder.transform(chunk['protein_name'].values.reshape(-1, 1))
    fingerprints_test = [convert_to_numpy_array(fp) for fp in chunk['fingerprints'].tolist()]
    X_test_final = [np.concatenate([fp, protein]) for fp, protein in zip(fingerprints_test, protein_onehot_test.tolist())]
    y_test_pred = model.predict(X_test_final)

    chunk['binds_pred'] = y_test_pred
    submission_chunks.append(chunk[['id', 'binds_pred']])

submission = pd.concat(submission_chunks, axis=0)
submission.to_csv('./submission.csv', index=False)